In [1]:
# -----------------------------------------------
# 🌐 TransFlip App with Splash Animation & Dark Mode
# -----------------------------------------------

import tkinter as tk
from PIL import Image, ImageTk
import time
from tkinter import ttk

# -----------------------------------------------
# 🖼️ Splash Screen with Fade-In, Typing Text, and Loader
# -----------------------------------------------
def show_splash():
    splash = tk.Tk()
    splash.title("Welcome")
    splash.geometry("1000x650")
    splash.configure(bg="#ffffff")
    splash.overrideredirect(True)

    # Load splash image
    try:
        img = Image.open("translator_logo.jpg")
        img = img.resize((1000, 650))
        img_tk = ImageTk.PhotoImage(img)
        label = tk.Label(splash, image=img_tk)
        label.image = img_tk
        label.place(x=0, y=0)
    except:
        label = tk.Label(splash, text="🌐 TransFlip App", font=("Helvetica", 30), bg="#ffffff")
        label.pack(expand=True)

    # Typing animation label
    typing_label = tk.Label(splash, text="", font=("Helvetica", 20, "italic"), bg="#ffffff", fg="#1a1a1a")
    typing_label.place(relx=0.5, rely=0.85, anchor="center")

    def type_text(text, i=0):
        if i < len(text):
            typing_label.config(text=text[:i+1])
            splash.after(100, lambda: type_text(text, i+1))

    type_text("Starting Real TransFlip App...")

    # Add loader bar
    progress = ttk.Progressbar(splash, orient="horizontal", length=400, mode="determinate")
    progress.place(relx=0.5, rely=0.93, anchor="center")

    def load_bar(value=0):
        if value <= 100:
            progress["value"] = value
            splash.after(30, lambda: load_bar(value + 2))
        else:
            splash.destroy()
            run_app()

    splash.attributes("-alpha", 0.0)

    def fade_in(alpha=0.0):
        if alpha < 1.0:
            splash.attributes("-alpha", alpha)
            splash.after(50, lambda: fade_in(alpha + 0.05))
        else:
            load_bar()

    fade_in()
    splash.mainloop()

# -----------------------------------------------
# 🌐 TransFlip App Window
# -----------------------------------------------
def run_app():
    import tkinter as tk
    from tkinter import ttk
    from deep_translator import GoogleTranslator
    from gtts import gTTS
    from playsound import playsound
    import speech_recognition as sr
    import os

    lang_code_map = {
        "English": "en",
        "Hindi": "hi",
        "Marathi": "mr",
        "Spanish": "es",
        "French": "fr",
        "German": "de",
        "Italian": "it",
        "Chinese (Simplified)": "zh-CN",
        "Japanese": "ja",
        "Korean": "ko",
        "Russian": "ru",
        "Arabic": "ar",
        "Portuguese": "pt",
        "Bengali": "bn",
        "Tamil": "ta",
        "Telugu": "te",
        "Gujarati": "gu",
        "Urdu": "ur",
        "Punjabi": "pa"
    }

    def speech_to_text():
        r = sr.Recognizer()
        with sr.Microphone() as source:
            input_text.delete("1.0", tk.END)
            audio = r.listen(source)
            try:
                text = r.recognize_google(audio)
                input_text.insert(tk.END, text)
            except sr.UnknownValueError:
                input_text.insert(tk.END, "Could not understand")
            except sr.RequestError:
                input_text.insert(tk.END, "Network error")

    def translate_text():
        source_lang = lang_code_map[from_lang.get()]
        target_lang = lang_code_map[to_lang.get()]
        original = input_text.get("1.0", tk.END).strip()

        if not original:
            return

        try:
            translated = GoogleTranslator(source=source_lang, target=target_lang).translate(original)
            output_text.delete("1.0", tk.END)
            output_text.insert(tk.END, translated)
            history_list.insert(tk.END, f"{original} → {translated}")

            if auto_speak.get():
                speak_text(translated, target_lang)

        except Exception as e:
            output_text.insert(tk.END, f"Error: {e}")

    def speak_text(text=None, lang=None):
        text = text or output_text.get("1.0", tk.END).strip()
        lang = lang or lang_code_map[to_lang.get()]
        if text:
            try:
                tts = gTTS(text=text, lang=lang)
                filename = "speak.mp3"
                tts.save(filename)
                playsound(filename)
                os.remove(filename)
            except Exception as e:
                print("Error:", e)

    def clear_all():
        input_text.delete("1.0", tk.END)
        output_text.delete("1.0", tk.END)
        history_list.delete(0, tk.END)

    def on_history_select(event):
        selection = history_list.curselection()
        if selection:
            line = history_list.get(selection[0])
            if "→" in line:
                original = line.split("→")[0].strip()
            else:
                original = line.strip()
            input_text.delete("1.0", tk.END)
            input_text.insert(tk.END, original)

    def toggle_theme():
        if theme_var.get():
            apply_dark_theme()
        else:
            apply_light_theme()

    def apply_light_theme():
        root.configure(bg="#f0faff")
        lang_frame.configure(bg="#f0faff")
        text_frame.configure(bg="#f0faff")
        button_frame.configure(bg="#f0faff")
        history_label.configure(bg="#f0faff", fg="black")

    def apply_dark_theme():
        root.configure(bg="#2e2e2e")
        lang_frame.configure(bg="#2e2e2e")
        text_frame.configure(bg="#2e2e2e")
        button_frame.configure(bg="#2e2e2e")
        history_label.configure(bg="#2e2e2e", fg="white")

    root = tk.Tk()
    root.title("🔄 TransFlip App")
    root.geometry("1000x650")
    root.configure(bg="#f0faff")

    theme_var = tk.BooleanVar(value=False)

    lang_frame = tk.Frame(root, bg="#f0faff")
    lang_frame.pack(pady=15)

    ttk.Label(lang_frame, text="From:").pack(side=tk.LEFT, padx=5)
    from_lang = ttk.Combobox(lang_frame, values=list(lang_code_map.keys()), state="readonly", width=14)
    from_lang.set("English")
    from_lang.pack(side=tk.LEFT, padx=5)

    ttk.Label(lang_frame, text="To:").pack(side=tk.LEFT, padx=5)
    to_lang = ttk.Combobox(lang_frame, values=list(lang_code_map.keys()), state="readonly", width=14)
    to_lang.set("Hindi")
    to_lang.pack(side=tk.LEFT, padx=5)

    auto_speak = tk.BooleanVar()
    auto_check = ttk.Checkbutton(lang_frame, text="🔊 Auto Speak", variable=auto_speak)
    auto_check.pack(side=tk.LEFT, padx=10)

    theme_switch = ttk.Checkbutton(lang_frame, text="🌙 Dark Mode", variable=theme_var, command=toggle_theme)
    theme_switch.pack(side=tk.LEFT, padx=10)

    text_frame = tk.Frame(root, bg="#f0faff")
    text_frame.pack(pady=10)

    input_text = tk.Text(text_frame, width=45, height=10, font=("Helvetica", 12))
    input_text.grid(row=0, column=0, padx=10)

    output_text = tk.Text(text_frame, width=45, height=10, font=("Helvetica", 12), fg="green")
    output_text.grid(row=0, column=1, padx=10)

    button_frame = tk.Frame(root, bg="#f0faff")
    button_frame.pack(pady=15)

    ttk.Button(button_frame, text="🎤 Speak", command=speech_to_text).grid(row=0, column=0, padx=10)
    ttk.Button(button_frame, text="🔁 Translate", command=translate_text).grid(row=0, column=1, padx=10)
    ttk.Button(button_frame, text="🔊 Speak Output", command=speak_text).grid(row=0, column=2, padx=10)
    ttk.Button(button_frame, text="🧹 Clear", command=clear_all).grid(row=0, column=3, padx=10)

    history_label = tk.Label(root, text="🕘 Translation History", font=("Helvetica", 12, "bold"), bg="#f0faff")
    history_label.pack(pady=5)

    history_list = tk.Listbox(root, width=110, height=6, font=("Helvetica", 10))
    history_list.pack()
    history_list.bind("<<ListboxSelect>>", on_history_select)

    apply_light_theme()
    root.mainloop()

# -----------------------------------------------
# 🚀 Launch the App
# -----------------------------------------------
show_splash()
